In [4]:
#imports
import numpy as np
import random
import matplotlib.pyplot as plt
import pandas
from PIL import Image
import os
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
import glob
import cv2
import time as tm
import keras_tuner as kt

In [5]:
def build_ANN(hp):
    #create model object
    model = Sequential()
    #adding first convolutional layer    
    model.add(Conv2D(
        #adding filter 
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        # adding filter size or kernel size
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        #activation function
        activation=hp.Choice("activation", ["relu", "tanh", "sigmoid"]),
        input_shape=X.shape[1:]))
    
    
    for i in range(hp.Int("num_layers", 1, 3)):
        #Adding nth convlutional layer
        model.add(Conv2D(
            #adding filter 
            filters=hp.Int('conv_2_filter', min_value=32, max_value=128, step=16),
            #adding filter size or kernel size
            kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
            #activation function
            activation='relu'))

        #added max pooling
        model.add(MaxPooling2D(pool_size=(2,2)))
        
        
        #added dropout 
        if hp.Boolean("dropout"):
            model.add(Dropout(rate=0.20))
    
    
    
    #adding flatten layer 
    model.add(Flatten())
        
    #adding fully connected layer
    model.add(Dense(units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16), activation='relu'))
    
    #adds final dropout layer
    if hp.Boolean("dropout"):
        model.add(Dropout(rate = .5))
    
    #output
    model.add(Dense(5, activation='softmax'))

    
    #chooses optimizer
    optimizer = hp.Choice('optimizer', ['adam', 'sgd' ])
    
    
    #compilation of model
    model.compile(optimizer, loss='categorical_crossentropy', metrics=['accuracy']) 
    #return model 
    return model

In [6]:
#buids model 
model = build_ANN(kt.HyperParameters())
#creates tuner
tuner = kt.RandomSearch(
    hypermodel=build_ANN,
    objective="val_accuracy",
    max_trials=10,
    executions_per_trial=5,
)
#gets best models 
tuner.search(train_X, train_y, epochs=3, validation_data=(test_X, test_y))

## folder path to save models in --- CHANGE AS NEEDED 
folder = '/Users/moises/Desktop/'

#gets the top 10 hyperparemeters
best_hyps = tuner.get_best_hyperparameters(10)

#gets the top 10 models, trains them, and saves them in a folder 
for i in range(len(best_hyps)):
    #get best ith hyper params 
    best_hp = best_hyps[i]
    #build model from hyperparamters 
    model=tuner.hypermodel.build(best_hp)
    #trains model
    model.fit(training_X, training_y, epochs=10, validation_split=0.1, initial_epoch=3)
    #gets model accuracy 
    _, accuracy = model.evaluate(test_X, test_y)
        
    #saves model in file 
    file_name = str(i) + str(accuracy)
    model.save(folder + file_name)

2022-05-16 17:26:16.851423: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


NameError: name 'X' is not defined

## Model saving

In [1]:
save_path = '[PUT SAVE PATH HERE]'

In [230]:
model.save(save_path)

2022-05-07 17:50:31.131774: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /Users/maxperozek/ML-CP341/8/saved-models/assets


In [231]:
loaded_model = keras.models.load_model(save_path)